In [1]:
import os
import tensorflow as tf
import time
import csv

In [2]:
def decode(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [3]:
batch_size=100

train_file_name = "Shuffled_PE92_Train.tfrecord"
test_file_name = "Shuffled_PE92_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

In [4]:

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


In [5]:
train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

200215


In [6]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [7]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])

is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)


In [8]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [9]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [10]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [11]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [12]:
W5 = tf.Variable(tf.random_normal([4*4*256, 512], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fc1 = tf.matmul(fc1, W5)
fc1 = tf.nn.relu(fc1)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 512), dtype=float32)


In [13]:
W6 = tf.Variable(tf.random_normal([512,2350],stddev=0.01))
logit = tf.matmul(fc1, W6)
print(logit)

Tensor("MatMul_1:0", shape=(?, 2350), dtype=float32)


In [14]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
loss_summ=tf.summary.scalar("Loss",loss)

In [15]:
prediction = tf.argmax(logit,1)
correct_prediction = tf.equal(prediction, input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [16]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    predict_file = open('./save/PE92/CNN/prediction.csv','w',encoding='utf-8',newline='')
    label_file = open('./save/PE92/CNN/labels.csv','w',encoding='utf-8',newline='')
    pred_writer = csv.writer(predict_file, delimiter=',')
    label_writer = csv.writer(label_file, delimiter=',')
    
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("./logs/CNN/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNN/PE92/Test")

    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    
    start_time = time.time()
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            summary, train_acc, train_loss=sess.run([merged_summary, accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
          
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
            
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %f" %(elapsed))
        
        
    print("Training is done")
    save_path = saver.save(sess, "save/PE92/CNN/CNN_PE92.ckpt")
    acc_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            test_acc=test_acc/len(batch_labels)
            print("Test acc is : %f"%(test_acc))
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    pred_writer.writerow(test_prediction)
    label_writer.writerow(batch_labels)
    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    predict_file.close()
    label_file.close()
    train_writer.close()
    test_writer.close()

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.020000, train loss: 7.623884
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 7.742446
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 8.501412
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 7.561935
Epoch :0 | step 400, training accuracy: 0.000000, train loss: 7.420147
Epoch :0 | step 500, training accuracy: 0.000000, train loss: 7.243591
Epoch :0 | step 600, training accuracy: 0.000000, train loss: 7.134609
Epoch :0 | step 700, training accuracy: 0.010000, train loss: 7.173365
Epoch :0 | step 800, training accuracy: 0.010000, train loss: 7.387272
Epoch :0 | step 900, training accuracy: 0.010000, train loss: 7.020967
Epoch :0 | step 1000, training accuracy: 0.010000, train loss: 7.167777
Epoch :0 | step 1100, training accuracy: 0.030000, train loss: 6.574766
Epoch :0 | step 1200, training accuracy: 0.000000, train loss: 6.624871
Epoch :0 | step 1300, training accuracy: 0.040000, t

Epoch :5 | step 600, training accuracy: 0.200000, train loss: 4.264464
Epoch :5 | step 700, training accuracy: 0.220000, train loss: 4.485802
Epoch :5 | step 800, training accuracy: 0.240000, train loss: 4.020646
Epoch :5 | step 900, training accuracy: 0.100000, train loss: 4.329251
Epoch :5 | step 1000, training accuracy: 0.120000, train loss: 4.315358
Epoch :5 | step 1100, training accuracy: 0.130000, train loss: 4.530685
Epoch :5 | step 1200, training accuracy: 0.190000, train loss: 4.374332
Epoch :5 | step 1300, training accuracy: 0.200000, train loss: 4.335814
Epoch :5 | step 1400, training accuracy: 0.240000, train loss: 4.079417
Epoch :5 | step 1500, training accuracy: 0.180000, train loss: 4.073099
Epoch :5 | step 1600, training accuracy: 0.190000, train loss: 4.337715
Epoch :5 | step 1700, training accuracy: 0.190000, train loss: 4.155288
Epoch :5 | step 1800, training accuracy: 0.220000, train loss: 4.102004
Epoch :5 | step 1900, training accuracy: 0.180000, train loss: 4.469

Epoch :10 | step 1200, training accuracy: 0.840000, train loss: 0.641124
Epoch :10 | step 1300, training accuracy: 0.800000, train loss: 0.698739
Epoch :10 | step 1400, training accuracy: 0.850000, train loss: 0.579197
Epoch :10 | step 1500, training accuracy: 0.940000, train loss: 0.461498
Epoch :10 | step 1600, training accuracy: 0.940000, train loss: 0.483619
Epoch :10 | step 1700, training accuracy: 0.900000, train loss: 0.496890
Epoch :10 | step 1800, training accuracy: 0.900000, train loss: 0.502252
Epoch :10 | step 1900, training accuracy: 0.860000, train loss: 0.539272
Epoch :10 | step 2000, training accuracy: 0.930000, train loss: 0.478390
Test Acc : 0.880000, Test Loss :0.492582
Time : 1789.918377
Epoch :11 | step 0, training accuracy: 0.850000, train loss: 0.592258
Epoch :11 | step 100, training accuracy: 0.900000, train loss: 0.466804
Epoch :11 | step 200, training accuracy: 0.910000, train loss: 0.527543
Epoch :11 | step 300, training accuracy: 0.910000, train loss: 0.4045

Epoch :15 | step 1600, training accuracy: 0.940000, train loss: 0.210802
Epoch :15 | step 1700, training accuracy: 0.960000, train loss: 0.161601
Epoch :15 | step 1800, training accuracy: 0.960000, train loss: 0.129445
Epoch :15 | step 1900, training accuracy: 0.970000, train loss: 0.193288
Epoch :15 | step 2000, training accuracy: 0.970000, train loss: 0.140806
Test Acc : 0.960000, Test Loss :0.175137
Time : 2598.384619
Epoch :16 | step 0, training accuracy: 0.950000, train loss: 0.165951
Epoch :16 | step 100, training accuracy: 0.970000, train loss: 0.074176
Epoch :16 | step 200, training accuracy: 0.980000, train loss: 0.090567
Epoch :16 | step 300, training accuracy: 0.990000, train loss: 0.086193
Epoch :16 | step 400, training accuracy: 0.980000, train loss: 0.078616
Epoch :16 | step 500, training accuracy: 0.990000, train loss: 0.088985
Epoch :16 | step 600, training accuracy: 0.970000, train loss: 0.075779
Epoch :16 | step 700, training accuracy: 0.950000, train loss: 0.134826
E

Epoch :20 | step 2000, training accuracy: 0.980000, train loss: 0.047991
Test Acc : 0.990000, Test Loss :0.033718
Time : 3403.815687
Epoch :21 | step 0, training accuracy: 0.980000, train loss: 0.043809
Epoch :21 | step 100, training accuracy: 0.970000, train loss: 0.103126
Epoch :21 | step 200, training accuracy: 0.980000, train loss: 0.149226
Epoch :21 | step 300, training accuracy: 0.940000, train loss: 0.459503
Epoch :21 | step 400, training accuracy: 1.000000, train loss: 0.018795
Epoch :21 | step 500, training accuracy: 0.980000, train loss: 0.076159
Epoch :21 | step 600, training accuracy: 0.990000, train loss: 0.043799
Epoch :21 | step 700, training accuracy: 0.980000, train loss: 0.040746
Epoch :21 | step 800, training accuracy: 0.980000, train loss: 0.074856
Epoch :21 | step 900, training accuracy: 0.980000, train loss: 0.054356
Epoch :21 | step 1000, training accuracy: 0.990000, train loss: 0.049690
Epoch :21 | step 1100, training accuracy: 0.970000, train loss: 0.059825
Epo

Epoch :26 | step 300, training accuracy: 0.990000, train loss: 0.039864
Epoch :26 | step 400, training accuracy: 1.000000, train loss: 0.012706
Epoch :26 | step 500, training accuracy: 1.000000, train loss: 0.012156
Epoch :26 | step 600, training accuracy: 0.980000, train loss: 0.067261
Epoch :26 | step 700, training accuracy: 0.990000, train loss: 0.053159
Epoch :26 | step 800, training accuracy: 0.990000, train loss: 0.030829
Epoch :26 | step 900, training accuracy: 1.000000, train loss: 0.009189
Epoch :26 | step 1000, training accuracy: 0.990000, train loss: 0.022731
Epoch :26 | step 1100, training accuracy: 0.980000, train loss: 0.041895
Epoch :26 | step 1200, training accuracy: 0.970000, train loss: 0.172979
Epoch :26 | step 1300, training accuracy: 1.000000, train loss: 0.019719
Epoch :26 | step 1400, training accuracy: 1.000000, train loss: 0.020448
Epoch :26 | step 1500, training accuracy: 0.980000, train loss: 0.069741
Epoch :26 | step 1600, training accuracy: 0.960000, train 

Epoch :31 | step 700, training accuracy: 1.000000, train loss: 0.002293
Epoch :31 | step 800, training accuracy: 1.000000, train loss: 0.009715
Epoch :31 | step 900, training accuracy: 1.000000, train loss: 0.005108
Epoch :31 | step 1000, training accuracy: 1.000000, train loss: 0.006733
Epoch :31 | step 1100, training accuracy: 0.990000, train loss: 0.025481
Epoch :31 | step 1200, training accuracy: 1.000000, train loss: 0.003316
Epoch :31 | step 1300, training accuracy: 0.990000, train loss: 0.010749
Epoch :31 | step 1400, training accuracy: 0.990000, train loss: 0.017153
Epoch :31 | step 1500, training accuracy: 0.980000, train loss: 0.049217
Epoch :31 | step 1600, training accuracy: 1.000000, train loss: 0.011619
Epoch :31 | step 1700, training accuracy: 0.990000, train loss: 0.012376
Epoch :31 | step 1800, training accuracy: 1.000000, train loss: 0.003325
Epoch :31 | step 1900, training accuracy: 1.000000, train loss: 0.001147
Epoch :31 | step 2000, training accuracy: 0.990000, tr

Epoch :36 | step 1100, training accuracy: 0.990000, train loss: 0.028200
Epoch :36 | step 1200, training accuracy: 1.000000, train loss: 0.000708
Epoch :36 | step 1300, training accuracy: 1.000000, train loss: 0.001099
Epoch :36 | step 1400, training accuracy: 1.000000, train loss: 0.002139
Epoch :36 | step 1500, training accuracy: 0.990000, train loss: 0.040198
Epoch :36 | step 1600, training accuracy: 0.990000, train loss: 0.017835
Epoch :36 | step 1700, training accuracy: 1.000000, train loss: 0.003834
Epoch :36 | step 1800, training accuracy: 1.000000, train loss: 0.002821
Epoch :36 | step 1900, training accuracy: 1.000000, train loss: 0.003541
Epoch :36 | step 2000, training accuracy: 0.990000, train loss: 0.015654
Test Acc : 1.000000, Test Loss :0.010179
Time : 5979.511009
Epoch :37 | step 0, training accuracy: 1.000000, train loss: 0.004973
Epoch :37 | step 100, training accuracy: 1.000000, train loss: 0.000996
Epoch :37 | step 200, training accuracy: 1.000000, train loss: 0.003

Epoch :41 | step 1500, training accuracy: 1.000000, train loss: 0.000614
Epoch :41 | step 1600, training accuracy: 1.000000, train loss: 0.000982
Epoch :41 | step 1700, training accuracy: 1.000000, train loss: 0.000460
Epoch :41 | step 1800, training accuracy: 1.000000, train loss: 0.004722
Epoch :41 | step 1900, training accuracy: 1.000000, train loss: 0.001523
Epoch :41 | step 2000, training accuracy: 1.000000, train loss: 0.000682
Test Acc : 1.000000, Test Loss :0.009235
Time : 6784.245037
Epoch :42 | step 0, training accuracy: 1.000000, train loss: 0.000722
Epoch :42 | step 100, training accuracy: 1.000000, train loss: 0.000940
Epoch :42 | step 200, training accuracy: 0.990000, train loss: 0.044484
Epoch :42 | step 300, training accuracy: 1.000000, train loss: 0.000855
Epoch :42 | step 400, training accuracy: 1.000000, train loss: 0.000584
Epoch :42 | step 500, training accuracy: 1.000000, train loss: 0.012249
Epoch :42 | step 600, training accuracy: 1.000000, train loss: 0.002018


Epoch :46 | step 1900, training accuracy: 1.000000, train loss: 0.000822
Epoch :46 | step 2000, training accuracy: 1.000000, train loss: 0.001121
Test Acc : 0.960000, Test Loss :0.125902
Time : 7589.171076
Epoch :47 | step 0, training accuracy: 0.990000, train loss: 0.027057
Epoch :47 | step 100, training accuracy: 1.000000, train loss: 0.000844
Epoch :47 | step 200, training accuracy: 1.000000, train loss: 0.000570
Epoch :47 | step 300, training accuracy: 1.000000, train loss: 0.004865
Epoch :47 | step 400, training accuracy: 1.000000, train loss: 0.002443
Epoch :47 | step 500, training accuracy: 1.000000, train loss: 0.013064
Epoch :47 | step 600, training accuracy: 1.000000, train loss: 0.001643
Epoch :47 | step 700, training accuracy: 1.000000, train loss: 0.000524
Epoch :47 | step 800, training accuracy: 0.990000, train loss: 0.071411
Epoch :47 | step 900, training accuracy: 1.000000, train loss: 0.003475
Epoch :47 | step 1000, training accuracy: 1.000000, train loss: 0.002715
Epo

In [19]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
test_dataset = test_dataset.repeat()
test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

total_test_batch = int(test_data_length/batch_size)+1

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types, test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

665890


In [21]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    predict_file = open('./save/HanDB/CNN/prediction.csv','w',encoding='utf-8',newline='')
    label_file = open('./save/HanDB/CNN/labels.csv','w',encoding='utf-8',newline='')
    pred_writer = csv.writer(predict_file, delimiter=',')
    label_writer = csv.writer(label_file, delimiter=',')
    
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("./logs/CNN/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNN/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)#
    #Train Model
    
    start_time = time.time()
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            sess.run(optimizer,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            summary, train_acc, train_loss=sess.run([merged_summary, accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0,is_training:False})
            train_writer.add_summary(summary,i)
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
          
        
            
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %f" %(elapsed))
        
        
    print("Training is done")
    save_path = saver.save(sess, "save\HanDB\CNN\CNN_HanDB.ckpt")
    acc_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy,loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            test_acc=test_acc/len(batch_labels)
            print("Test acc is : %f"%(test_acc))
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    pred_writer.writerow(test_prediction)
    label_writer.writerow(batch_labels)
    
    #Test Model
    coord.request_stop()
    coord.join(threads)
    predict_file.close()
    label_file.close()
    train_writer.close()
    test_writer.close()

Train total Batch :6658
Epoch :0 | step 0, training accuracy: 0.020000, train loss: 7.622694
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 7.728064
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 8.102530
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 7.778723
Epoch :0 | step 400, training accuracy: 0.010000, train loss: 7.735517
Epoch :0 | step 500, training accuracy: 0.030000, train loss: 5.677297
Epoch :0 | step 600, training accuracy: 0.250000, train loss: 4.383336
Epoch :0 | step 700, training accuracy: 0.040000, train loss: 7.481074
Epoch :0 | step 800, training accuracy: 0.100000, train loss: 6.192692
Epoch :0 | step 900, training accuracy: 0.150000, train loss: 5.003740
Epoch :0 | step 1000, training accuracy: 0.410000, train loss: 3.603528
Epoch :0 | step 1100, training accuracy: 0.190000, train loss: 4.432267
Epoch :0 | step 1200, training accuracy: 0.230000, train loss: 4.328988
Epoch :0 | step 1300, training accuracy: 0.310000, t

Epoch :1 | step 4700, training accuracy: 0.920000, train loss: 0.502876
Epoch :1 | step 4800, training accuracy: 0.870000, train loss: 0.639548
Epoch :1 | step 4900, training accuracy: 0.900000, train loss: 0.361670
Epoch :1 | step 5000, training accuracy: 0.860000, train loss: 0.634806


KeyboardInterrupt: 

In [ ]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels